In [52]:
import findspark
import json
import os
import functools
import tempfile
import requests
import pathlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, path, glob
import numpy as np
import re

from pyspark.sql.functions import lit
from pyspark               import SparkContext
from pyspark.streaming     import StreamingContext
from pyspark.sql           import SQLContext, SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types     import *
from pyspark.sql           import functions as F
from datetime import datetime as dt
from uuid import uuid4
from requests_oauthlib import OAuth1Session
from collections import namedtuple
from IPython.display import display, clear_output
from functools import reduce  

In [124]:
# del df00, df01

In [125]:
json_dir = 'D:/twitterdata/twitter4/'

json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

In [126]:
dfs = []

In [127]:
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
    dfs.append(json_data)
df00 = pd.concat(dfs)

In [128]:
len(df00)

9999

In [129]:
df00.columns.tolist()

['created_at',
 'id',
 'id_str',
 'text',
 'display_text_range',
 'source',
 'truncated',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'in_reply_to_screen_name',
 'geo',
 'coordinates',
 'place',
 'contributors',
 'is_quote_status',
 'quote_count',
 'reply_count',
 'retweet_count',
 'favorite_count',
 'favorited',
 'retweeted',
 'possibly_sensitive',
 'filter_level',
 'lang',
 'timestamp_ms',
 'user.id',
 'user.id_str',
 'user.name',
 'user.screen_name',
 'user.location',
 'user.url',
 'user.description',
 'user.translator_type',
 'user.protected',
 'user.verified',
 'user.followers_count',
 'user.friends_count',
 'user.listed_count',
 'user.favourites_count',
 'user.statuses_count',
 'user.created_at',
 'user.utc_offset',
 'user.time_zone',
 'user.geo_enabled',
 'user.lang',
 'user.contributors_enabled',
 'user.is_translator',
 'user.profile_background_color',
 'user.profile_background_image_url',
 'user.profile_backgroun

In [130]:
df00[df00['user.screen_name'] == 'Nookerloveu2']['timestamp_ms']

Series([], Name: timestamp_ms, dtype: object)

In [131]:
df01 = df00[['created_at','id','text','timestamp_ms','user.screen_name','user.location','user.followers_count','user.friends_count','user.favourites_count','user.created_at']]

In [132]:
df01['hashtags'] =  df01['text'].apply(lambda x: re.findall(r'#[a-zA-Z]+', x))

<ipython-input-132-218e500a185c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df01['hashtags'] =  df01['text'].apply(lambda x: re.findall(r'#[a-zA-Z]+', x))


In [133]:
df01

,created_at,id,text,timestamp_ms,user.screen_name,user.location,user.followers_count,user.friends_count,user.favourites_count,user.created_at,hashtags
0,Fri Aug 27 09:55:55 +0000 2021,1431193751230160899,What do you think?\n#digitalmarketing #Ronaldo...,1630058155244,ummekulsum47,United Kingdom,676,993,1114,Fri Jun 11 17:15:52 +0000 2021,"[#digitalmarketing, #Ronaldo, #Sunshine, #soci..."
0,Fri Aug 27 09:55:55 +0000 2021,1431193751939162114,RT @goal: Ronaldo has left Juventus's training...,1630058155413,Makintamie,Nigeria🇳🇬,5067,2877,2663,Sun Feb 13 21:13:52 +0000 2011,[]
0,Fri Aug 27 09:55:55 +0000 2021,1431193753142841347,RT @ESPNFC: Man United fans following the Rona...,1630058155700,drew_addei,None,3115,4987,258445,Sun Jan 11 15:01:15 +0000 2015,[]
0,Fri Aug 27 09:55:55 +0000 2021,1431193753289641984,RT @Priceless_Silva: Not everything has to be ...,1630058155735,gIyn_,"North West, England",283,219,108505,Sat Sep 16 18:40:11 +0000 2017,[]
0,Fri Aug 27 09:55:55 +0000 2021,1431193754116005895,RT @Football__Tweet: Sir Alex Ferguson waiting...,1630058155932,redview1975,MANCHESTER/UK,1120,1170,96882,Thu Apr 07 14:55:22 +0000 2011,[]
...,...,...,...,...,...,...,...,...,...,...,...
0,Fri Aug 27 10:09:43 +0000 2021,1431197226970370049,"RT @Nigerianscamsss: Man Utd fanbase is funny,...",1630058983925,tombradyyisgoat,In a red 64 Impala,460,145,562,Sun Oct 25 10:37:26 +0000 2015,[]
0,Fri Aug 27 10:09:44 +0000 2021,1431197227523842048,RT @GreatWhiteNueve: Ronaldo: I wanna join Uni...,1630058984057,PERlZlNAN,None,297,567,120968,Sun Nov 11 00:09:20 +0000 2018,[]
0,Fri Aug 27 10:09:44 +0000 2021,1431197227712716807,RT @GreatWhiteNueve: Ronaldo: I wanna join Uni...,1630058984102,JuliusAce5,center of the earth,2077,2074,133156,Fri Oct 05 17:23:20 +0000 2018,[]
0,Fri Aug 27 10:09:44 +0000 2021,1431197228455055361,RT @TransferChecker: I genuinely think Ronaldo...,1630058984279,Ilyas_ali10,None,21,37,42709,Sat Nov 23 12:41:59 +0000 2019,[]


In [134]:
df01['addname'] =  df01['text'].apply(lambda x: re.findall(r'@[a-zA-Z]+', x))

<ipython-input-134-b4f29abc9e59>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df01['addname'] =  df01['text'].apply(lambda x: re.findall(r'@[a-zA-Z]+', x))


In [135]:
df01

,created_at,id,text,timestamp_ms,user.screen_name,user.location,user.followers_count,user.friends_count,user.favourites_count,user.created_at,hashtags,addname
0,Fri Aug 27 09:55:55 +0000 2021,1431193751230160899,What do you think?\n#digitalmarketing #Ronaldo...,1630058155244,ummekulsum47,United Kingdom,676,993,1114,Fri Jun 11 17:15:52 +0000 2021,"[#digitalmarketing, #Ronaldo, #Sunshine, #soci...",[]
0,Fri Aug 27 09:55:55 +0000 2021,1431193751939162114,RT @goal: Ronaldo has left Juventus's training...,1630058155413,Makintamie,Nigeria🇳🇬,5067,2877,2663,Sun Feb 13 21:13:52 +0000 2011,[],[@goal]
0,Fri Aug 27 09:55:55 +0000 2021,1431193753142841347,RT @ESPNFC: Man United fans following the Rona...,1630058155700,drew_addei,None,3115,4987,258445,Sun Jan 11 15:01:15 +0000 2015,[],[@ESPNFC]
0,Fri Aug 27 09:55:55 +0000 2021,1431193753289641984,RT @Priceless_Silva: Not everything has to be ...,1630058155735,gIyn_,"North West, England",283,219,108505,Sat Sep 16 18:40:11 +0000 2017,[],[@Priceless]
0,Fri Aug 27 09:55:55 +0000 2021,1431193754116005895,RT @Football__Tweet: Sir Alex Ferguson waiting...,1630058155932,redview1975,MANCHESTER/UK,1120,1170,96882,Thu Apr 07 14:55:22 +0000 2011,[],[@Football]
...,...,...,...,...,...,...,...,...,...,...,...,...
0,Fri Aug 27 10:09:43 +0000 2021,1431197226970370049,"RT @Nigerianscamsss: Man Utd fanbase is funny,...",1630058983925,tombradyyisgoat,In a red 64 Impala,460,145,562,Sun Oct 25 10:37:26 +0000 2015,[],[@Nigerianscamsss]
0,Fri Aug 27 10:09:44 +0000 2021,1431197227523842048,RT @GreatWhiteNueve: Ronaldo: I wanna join Uni...,1630058984057,PERlZlNAN,None,297,567,120968,Sun Nov 11 00:09:20 +0000 2018,[],[@GreatWhiteNueve]
0,Fri Aug 27 10:09:44 +0000 2021,1431197227712716807,RT @GreatWhiteNueve: Ronaldo: I wanna join Uni...,1630058984102,JuliusAce5,center of the earth,2077,2074,133156,Fri Oct 05 17:23:20 +0000 2018,[],[@GreatWhiteNueve]
0,Fri Aug 27 10:09:44 +0000 2021,1431197228455055361,RT @TransferChecker: I genuinely think Ronaldo...,1630058984279,Ilyas_ali10,None,21,37,42709,Sat Nov 23 12:41:59 +0000 2019,[],[@TransferChecker]


In [136]:
df01.reset_index(inplace=True)

In [137]:
df01 = df01.drop(columns=['index'])

In [139]:
for i in range(1,200,1):    
    print(df01['hashtags'][i])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['#MCFC']
[]
[]
[]
['#UCL']
[]
[]
[]
[]
[]
[]
[]
[]
['#ManCity']
[]
[]
[]
[]
[]
[]
[]
[]
['#MUFC']
[]
['#ManCity']
[]
[]
[]
[]
[]
[]
[]
['#UCL']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['#ManCity']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['#CristianoRonaldo', '#ManchesterCity', '#CR', '#Juventus']
[]
[]
['#mcfc', '#Ronaldo']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['#ManCity']
[]
[]
[]
[]
[]
[]
['#MUFC']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['#UCL']
[]
[]
[]
['#Juventus']
[]
[]
['#CristianoRonaldo', '#ManchesterCity', '#MCFC']
[]
[]


In [140]:
df01.columns

Index(['created_at', 'id', 'text', 'timestamp_ms', 'user.screen_name',
       'user.location', 'user.followers_count', 'user.friends_count',
       'user.favourites_count', 'user.created_at', 'hashtags', 'addname'],
      dtype='object')

In [141]:
df01 = df01.rename(columns={'user.screen_name': 'userscreenname', \
                       'user.location': 'userlocation',  \
                       'user.followers_count': 'userfollowerscount', 'user.friends_count': 'userfriendscount', \
                       'user.favourites_count': 'userfavouritescount', 'user.created_at': 'usercreatedat'})

In [142]:
df01

,created_at,id,text,timestamp_ms,userscreenname,userlocation,userfollowerscount,userfriendscount,userfavouritescount,usercreatedat,hashtags,addname
0,Fri Aug 27 09:55:55 +0000 2021,1431193751230160899,What do you think?\n#digitalmarketing #Ronaldo...,1630058155244,ummekulsum47,United Kingdom,676,993,1114,Fri Jun 11 17:15:52 +0000 2021,"[#digitalmarketing, #Ronaldo, #Sunshine, #soci...",[]
1,Fri Aug 27 09:55:55 +0000 2021,1431193751939162114,RT @goal: Ronaldo has left Juventus's training...,1630058155413,Makintamie,Nigeria🇳🇬,5067,2877,2663,Sun Feb 13 21:13:52 +0000 2011,[],[@goal]
2,Fri Aug 27 09:55:55 +0000 2021,1431193753142841347,RT @ESPNFC: Man United fans following the Rona...,1630058155700,drew_addei,None,3115,4987,258445,Sun Jan 11 15:01:15 +0000 2015,[],[@ESPNFC]
3,Fri Aug 27 09:55:55 +0000 2021,1431193753289641984,RT @Priceless_Silva: Not everything has to be ...,1630058155735,gIyn_,"North West, England",283,219,108505,Sat Sep 16 18:40:11 +0000 2017,[],[@Priceless]
4,Fri Aug 27 09:55:55 +0000 2021,1431193754116005895,RT @Football__Tweet: Sir Alex Ferguson waiting...,1630058155932,redview1975,MANCHESTER/UK,1120,1170,96882,Thu Apr 07 14:55:22 +0000 2011,[],[@Football]
...,...,...,...,...,...,...,...,...,...,...,...,...
9994,Fri Aug 27 10:09:43 +0000 2021,1431197226970370049,"RT @Nigerianscamsss: Man Utd fanbase is funny,...",1630058983925,tombradyyisgoat,In a red 64 Impala,460,145,562,Sun Oct 25 10:37:26 +0000 2015,[],[@Nigerianscamsss]
9995,Fri Aug 27 10:09:44 +0000 2021,1431197227523842048,RT @GreatWhiteNueve: Ronaldo: I wanna join Uni...,1630058984057,PERlZlNAN,None,297,567,120968,Sun Nov 11 00:09:20 +0000 2018,[],[@GreatWhiteNueve]
9996,Fri Aug 27 10:09:44 +0000 2021,1431197227712716807,RT @GreatWhiteNueve: Ronaldo: I wanna join Uni...,1630058984102,JuliusAce5,center of the earth,2077,2074,133156,Fri Oct 05 17:23:20 +0000 2018,[],[@GreatWhiteNueve]
9997,Fri Aug 27 10:09:44 +0000 2021,1431197228455055361,RT @TransferChecker: I genuinely think Ronaldo...,1630058984279,Ilyas_ali10,None,21,37,42709,Sat Nov 23 12:41:59 +0000 2019,[],[@TransferChecker]


In [143]:
for idx, group in df01.groupby(np.arange(len(df01))//1): 
    group.to_json(f'D:\\twitterdata\\tw\\{idx}_04name.json', orient='records') 